In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [40]:
plt.rcParams.update({"text.usetex": False,
                     "font.size": 20,
                     "pdf.fonttype": 42,
                     "ps.fonttype": 42})

In [41]:
target_latency = 150

In [42]:
local_df = pd.read_csv("../measurements/local_long/map.csv")

root_folder = "../measurements/static_test_50/"
mag_folder = "../measurements/real_bw_test_15fps/"

map_df = pd.read_csv(root_folder+"map.csv")
cha_df = pd.read_csv(root_folder+"characteristic.csv")
mag_df = pd.read_csv(mag_folder+"manager.csv")
tim_df = pd.read_csv(root_folder+"time.csv")

cal_time = []
for i in range(len(tim_df)):
    if tim_df["technique"][i] == 2:
        cal_time.append(tim_df["compression_time"][i]/32)
    elif tim_df["technique"][i] == 3:
        cal_time.append(tim_df["compression_time"][i]-50)
    else:
        cal_time.append(tim_df["compression_time"][i])
tim_df["compression_time"] = cal_time

tim_df["transmission_time"] = cha_df["datasize_est"]*8/cha_df["consumed_bw"]*1000
tim_df["overall_est"] = tim_df["model_head_time"]*10+tim_df["model_tail_time"]+tim_df["framework_head_time"]+tim_df["framework_tail_time"] + tim_df["compression_time"]+tim_df["decompression_time"]+ tim_df["transmission_time"]

In [43]:
drops = []

for i in range(400):
    try:
        d = mag_df[mag_df["frame_id"]==i+1]["drop"].tolist()[0]
        if d >0.4:
            d=0.4
        drops.append(d)
    except:
        drops.append(drops[-1])


df = pd.DataFrame()
df["traget_drops"] = drops

In [44]:
df["map_drop"] = local_df["map"] - map_df["map"]
df["sen_drop"] = local_df["sensitivity"] - map_df["sensitivity"]

In [45]:
df["bandwidth"] = cha_df["bandwidth"]
df["bw_com"] = cha_df["consumed_bw"]
df["overall_time"] = tim_df["overall_est"]

In [46]:
df = df[200:]

In [47]:
df

,traget_drops,map_drop,sen_drop,bandwidth,bw_com,overall_time
200,0.4,0.166667,0.333333,8880000.0,1.513835e+06,167.523438
201,0.4,0.395833,0.500000,8880000.0,1.610682e+06,166.696230
202,0.4,0.250000,0.400000,8880000.0,1.679294e+06,166.410953
203,0.4,0.250000,0.200000,8880000.0,1.836847e+06,166.470343
204,0.4,0.250000,0.250000,8880000.0,1.828094e+06,168.161351
...,...,...,...,...,...,...
395,0.4,0.450000,0.428571,6000000.0,1.348941e+06,166.702121
396,0.4,0.262500,0.428571,6000000.0,1.338776e+06,166.527721
397,0.4,0.533333,0.571429,6000000.0,1.338918e+06,166.860008
398,0.4,0.400000,0.400000,6000000.0,1.330588e+06,166.359405


In [48]:
good_df = df[df["map_drop"]<df["traget_drops"]]
good_df= good_df[good_df["sen_drop"]<good_df["traget_drops"]]
good_df= good_df[good_df["bw_com"]<good_df["bandwidth"]]
# good_df= good_df[good_df["overall_time"]<target_latency]

In [49]:
len(good_df)/200

0.43